In [ ]:
import sys, os

sys.path.append("./pyFM/")  # My code builds on the pyFM.mesh.TriMesh class
sys.path.append("./VisualizationTools/")  # This is just for visualization purposes

In [ ]:
import dense_mesh as dm
import VisualizationTools as plu

from pyFM.mesh import TriMesh

# 1 - Computing the approximate spectrum for a dense mesh

In [ ]:
# Load mesh, normalize and center for simplicity
mesh1 = TriMesh("./data/40.off", area_normalize=True, center=True).process(
    k=0, intrinsic=True
)
mesh1.n_vertices

In [ ]:
# Define parameters for the process
process_params = {
    "dist_ratio": 3,  # rho = dist_ratio * average_radius
    "self_weight_limit": 0.25,  # Minimum value for self weight
    "correct_dist": False,
    "interpolation": "poly",
    "return_dist": True,
    "adapt_radius": True,
    "update_sample": True,
    "n_jobs": 10,
    "n_clusters": 5,
    "force_n_samples": False,
    "verbose": True,
}
n_samples = 2000

# Check the docstring for the meaning of each parameter
print(dm.process_mesh.__doc__)

In [ ]:
# Computes the approximate spectrum, Wb is used for \overline{W} and Ab for \overline{A}
# Evects1 are values at samples of sub1, distmat1 is the distance matrix
# Dense N1 x K eigenvector matrix can be obtained with U1 @ evects1
U1, Ab1, Wb1, sub1, distmat1 = dm.process_mesh(mesh1, n_samples, **process_params)
evals1, evects1 = dm.get_approx_spectrum(Wb1, Ab1, verbose=True)

In [ ]:
# Visualize the spectrum as values at the samples of sub1
plu.plot(mesh1, points=sub1, cmap_p=evects1[:, 1], colormap="coolwarm")

In [ ]:
# You can also check the "dense" eigenvector
plu.plot(mesh1, cmap=U1 @ evects1[:, 1], colormap="coolwarm")

# 2 - Shape Matching using approximate spectrum

In [ ]:
# Let's first load the meshes

mesh1 = TriMesh("./data/13.off", area_normalize=True, center=True).process(
    k=0, intrinsic=True
)
mesh2 = TriMesh("./data/42.off", area_normalize=True, center=True).process(
    k=0, intrinsic=True
)

plu.double_plot(mesh1, mesh2)

In [ ]:
# Define parameters for the process
process_params = {
    "dist_ratio": 3,  # rho = dist_ratio * average_radius
    "self_weight_limit": 0.25,  # Minimum value for self weight
    "correct_dist": False,
    "interpolation": "poly",
    "return_dist": True,
    "adapt_radius": True,
    "update_sample": True,
    "n_jobs": 10,
    "force_n_samples": False,
    "verbose": True,
}


n_samples = 2000

In [ ]:
U1, Ab1, Wb1, sub1, distmat1 = dm.process_mesh(mesh1, n_samples, **process_params)
evals1, evects1 = dm.get_approx_spectrum(Wb1, Ab1, verbose=True)

U2, Ab2, Wb2, sub2, distmat2 = dm.process_mesh(mesh2, n_samples, **process_params)
evals2, evects2 = dm.get_approx_spectrum(Wb2, Ab2, verbose=True)

In [ ]:
import pyFM.spectral as spectral
from pyFM.spectral.nn_utils import knn_query
from pyFM.refine.zoomout import zoomout_refine

import matplotlib.pyplot as plt

In [ ]:
# Compute an initial approximate functional map
p2p_21_sub_init = knn_query(mesh1.vertices[sub1], mesh2.vertices[sub2], k=1, n_jobs=10)
# We compute the initial functional map using the approximate spectrum here (same method that will be used inside ZoomOut)
FM_12_init = spectral.p2p_to_FM(
    p2p_21_sub_init, evects1[:, :20], evects2[:, :20], A2=Ab2
)

plt.imshow(FM_12_init, cmap="coolwarm")

In [ ]:
# You can perform ZoomOut like if you had resampled the whole mesh. This gives you a funcitonal map and a point-to-point map between the two samples (not the whole meshes)
FM_12_zo, p2p_21_sub_zo = zoomout_refine(
    FM_12_init,
    evects1,
    evects2,
    nit=16,
    step=5,
    A2=Ab2,
    return_p2p=True,
    n_jobs=10,
    verbose=True,
)

In [ ]:
# If you need a dense point-to-point map, you can use FM_12_zo as the functional map between the dense shapes. If needed, an accelerated version of this is described in the paper, but I found that its implementation is very machine-dependant.
p2p_21_zo = spectral.FM_to_p2p(FM_12_zo, U1 @ evects1, U2 @ evects2, n_jobs=10)

In [ ]:
plu.plot_p2p(mesh1, mesh2, p2p_21_zo, pretty=True)